<h2> PROYECTO - Aprendizaje No Supervisado<h2>

<h2>Integrantes:<h2>
<h3>- Maximiliano Zapater Cornejo<h3>
<h3>- Javiera Vukasovic Figueroa<h3>
<h3>- Sebastián Silva Espinoza<h3>
<h3>- Glen Restrepo Arteaga<h3>
<h3>- Marco Gutiérrez Corvalán<h3>

Utilizaremos un dataset para estimación de los niveles de obesidad según los hábitos alimenticios y condiciones físicas en 2111 personas de Colombia, Perú y México. 

**El dataset contiene los siguientes atributos**:

* Gender (*binario*): Género [*Female/Male*]

* Age (*numérico continuo*): Edad 
* Height (*numérico continuo*): Altura [*metros*]
* Weight (*numérico continuo)*: Peso [*kilogramos*]
* Family history with overweight (*binario*): ¿Algún miembro de la familia ha sufrido o sufre de sobrepeso?
* FAVC (*binario*): ¿Comes alimentos altos en calorías frecuentemente?
* FCVC (*numérico entero*): ¿Usualmente comes vegetales en tus comidas principales? [*Never/Sometimes/Always*]
* NCP (*numérico continuo*): ¿Cuántas comidas principales consumes diariamente? [*Between 1 and 2/Three/More than three*]
* CAEC (*categórica*): ¿Comes algún alimento entre comidas principales? [*No/Sometimes/Frequently/Always*]
* SMOKE (*binario*): ¿Fumas?
* CH2O (*numérico continuo*): ¿Cuánta agua bebes diariamente? [*Less than a liter/Between 1 and 2L/More than 2L*]
* SCC (*binario*): ¿Monitoreas las calorías que consumes diariamente?
* FAF (*numérico continuo*): ¿Qué tan seguido realizas actividad física? [*I do not have/1 or 2 days/2 or 4 days/4 or 5 days*]
* TUE (*numérico entero*): ¿Cuánto tiempo usas aparatos electrónicos tales como celular, videojuegos, televisión, computadora u otros?
* CALC (*categórica*): ¿Qué tan seguido bebes alcohol? [*I do not drink/Sometimes/Frequently/Always*]
* MTRANS (*categórica*): ¿Qué medio de transporte sueles utilizar? [*Automobile/Motorbike/Bike/Public Transportation/Walking*]

<h3> Preprocesamiento <h3>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import scale

warnings.filterwarnings('ignore')

df_obesity_raw = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
df_obesity = df_obesity_raw.drop(columns = ['NObeyesdad'])
display(df_obesity.head())

El dataset tiene atributos numéricos y categóricos. Para generar un modelo con un dataset de este estilo es necesario codificar las variables categóricas transformándolas en variables numéricas.

In [3]:
## One Hot Encoding
oh_enc = pd.get_dummies(df_obesity[['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']])
oh_enc = oh_enc.astype(int)

df_obesity_oh_enc = df_obesity.copy()
df_obesity_oh_enc.drop(columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS'], inplace = True)
df_obesity_oh_enc = pd.concat([df_obesity_oh_enc, oh_enc], axis = 1)

## Escalamos los datos
df_obesity_oh_enc_scaled = pd.DataFrame(scale(df_obesity_oh_enc))
df_obesity_oh_enc_scaled.columns = df_obesity_oh_enc.columns

In [4]:
## Frequency Encoding
df_obesity_freq_enc = df_obesity.copy()

## Lista de columnas categóricas
cat_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']

## Codificación por frecuencia
for col in cat_cols:
    freq = df_obesity_freq_enc[col].value_counts(normalize = True)  ## Frecuencia relativa
    df_obesity_freq_enc[col + '_freq'] = df_obesity_freq_enc[col].map(freq)

## Eliminar las columnas categóricas originales
df_obesity_freq_enc.drop(columns = cat_cols, inplace = True)

## Escalado de datos
df_obesity_freq_enc_scaled = pd.DataFrame(scale(df_obesity_freq_enc))
df_obesity_freq_enc_scaled.columns = df_obesity_freq_enc.columns

In [5]:
## Label Encoding
from sklearn.preprocessing import LabelEncoder
df_obesity_lab_enc = df_obesity.copy()

## Lista de columnas categóricas
cat_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']

for col in cat_cols:
    label_enc = LabelEncoder()
    df_obesity_lab_enc[col + '_lab_enc'] = label_enc.fit_transform(df_obesity_lab_enc[col])

## Eliminamos columnas categóricas iniciales
df_obesity_lab_enc.drop(columns=cat_cols, inplace=True)
df_obesity_lab_enc.head()

## Escalado de datos
df_obesity_lab_enc_scaled = pd.DataFrame(scale(df_obesity_lab_enc))
df_obesity_lab_enc_scaled.columns = df_obesity_lab_enc.columns

In [ ]:
## PCA con One Hot Encoding
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pca_pipe_oh = make_pipeline(StandardScaler(), PCA())
pca_pipe_oh.fit(df_obesity_oh_enc)

df_obesity_oh_pca = pca_pipe_oh.named_steps['pca']

n_componentes = np.arange(df_obesity_oh_pca.n_components_) + 1
varianza_expl = df_obesity_oh_pca.explained_variance_ratio_

## Determinamos el porcentaje de varianza explicada por las distintas componentes principales y graficamos
n_componentes = np.arange(df_obesity_oh_pca.n_components_) + 1
varianza_expl = df_obesity_oh_pca.explained_variance_ratio_

plt.figure(figsize = (12, 4))
plt.bar(x = n_componentes, height = varianza_expl, edgecolor = 'black')
plt.xticks(ticks = n_componentes)
for i in range(len(n_componentes)):
    plt.text(x = n_componentes[i], y = varianza_expl[i] + 0.01, s = f'{varianza_expl[i]:.2f}', ha = 'center')
    
plt.title('Porcentaje de varianza explicada vs Componente Principal (One Hot Encoding)')
plt.ylabel('Porcentaje de varianza explicada')
plt.xlabel('Componente Principal')
plt.ylim([0, 0.2])
plt.tight_layout()
plt.show()

In [ ]:
## Determinamos el porcentaje de la varianza explicada acumulada
varianza_expl_acum = df_obesity_oh_pca.explained_variance_ratio_.cumsum()

plt.figure(figsize = (12, 5))
plt.plot(n_componentes, varianza_expl_acum, marker = 'o', color = 'blue')
plt.plot(range(1, len(varianza_expl_acum)+1), varianza_expl_acum, marker = 'o')
plt.axhline(y = 0.8, color = 'r', linestyle = '--', label = '80% de varianza')
plt.title('Porcentaje de varianza explicada acumulada vs Número de Componentes Principales')
plt.ylabel('Porcentaje de varianza explicada acumulada')
plt.xlabel('Número de componentes principales')
for i in range(len(n_componentes)):
    plt.text(x = n_componentes[i], y = varianza_expl_acum[i] + 0.01, s = f'{varianza_expl_acum[i]:.2f}', ha = 'right')

plt.xticks(ticks = n_componentes)
plt.legend()
plt.grid()
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

In [ ]:
## PCA con Frequency Encoding
pca_pipe_freq = make_pipeline(StandardScaler(), PCA())
pca_pipe_freq.fit(df_obesity_freq_enc)
df_obesity_freq_pca = pca_pipe_freq.named_steps['pca']

n_componentes = np.arange(df_obesity_freq_pca.n_components_) + 1
varianza_expl = df_obesity_freq_pca.explained_variance_ratio_

## Determinamos el porcentaje de varianza explicada por las distintas componentes principales y graficamos
n_componentes = np.arange(df_obesity_freq_pca.n_components_) + 1
varianza_expl = df_obesity_freq_pca.explained_variance_ratio_

plt.figure(figsize = (12, 4))
plt.bar(x = n_componentes, height = varianza_expl, edgecolor = 'black')
plt.xticks(ticks = n_componentes)
for i in range(len(n_componentes)):
    plt.text(x = n_componentes[i], y = varianza_expl[i] + 0.01, s = f'{varianza_expl[i]:.2f}', ha = 'center')
    
plt.title('Porcentaje de varianza explicada vs Componente Principal (Frequency Encoded)')
plt.ylabel('Porcentaje de varianza explicada')
plt.xlabel('Componente Principal')
plt.ylim([0, 0.2])
plt.tight_layout()
plt.show()

In [ ]:
## Determinamos el porcentaje de la varianza explicada acumulada
varianza_expl_acum = df_obesity_freq_pca.explained_variance_ratio_.cumsum()

plt.figure(figsize = (12, 5))
plt.plot(n_componentes, varianza_expl_acum, marker = 'o', color = 'blue')
plt.plot(range(1, len(varianza_expl_acum)+1), varianza_expl_acum, marker = 'o')
plt.axhline(y = 0.8, color = 'r', linestyle = '--', label = '80% de varianza')
plt.title('Porcentaje de varianza explicada acumulada vs Número de Componentes Principales')
plt.ylabel('Porcentaje de varianza explicada acumulada')
plt.xlabel('Número de componentes principales')
for i in range(len(n_componentes)):
    plt.text(x = n_componentes[i], y = varianza_expl_acum[i] + 0.01, s = f'{varianza_expl_acum[i]:.2f}', ha = 'right')

plt.xticks(ticks = n_componentes)
plt.legend()
plt.grid()
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

In [ ]:
## PCA con Label Encoding
pca_pipe_lab = make_pipeline(StandardScaler(), PCA())
pca_pipe_lab.fit(df_obesity_lab_enc)
df_obesity_lab_pca = pca_pipe_lab.named_steps['pca']

n_componentes = np.arange(df_obesity_lab_pca.n_components_) + 1
varianza_expl = df_obesity_lab_pca.explained_variance_ratio_

## Determinamos el porcentaje de varianza explicada por las distintas componentes principales y graficamos
n_componentes = np.arange(df_obesity_lab_pca.n_components_) + 1
varianza_expl = df_obesity_lab_pca.explained_variance_ratio_

plt.figure(figsize = (12, 4))
plt.bar(x = n_componentes, height = varianza_expl, edgecolor = 'black')
plt.xticks(ticks = n_componentes)
for i in range(len(n_componentes)):
    plt.text(x = n_componentes[i], y = varianza_expl[i] + 0.01, s = f'{varianza_expl[i]:.2f}', ha = 'center')
    
plt.title('Porcentaje de varianza explicada vs Componente Principal (Label Encoding)')
plt.ylabel('Porcentaje de varianza explicada')
plt.xlabel('Componente Principal')
plt.ylim([0, 0.2])
plt.tight_layout()
plt.show()

In [ ]:
## Determinamos el porcentaje de la varianza explicada acumulada
varianza_expl_acum = df_obesity_lab_pca.explained_variance_ratio_.cumsum()

plt.figure(figsize = (12, 5))
plt.plot(n_componentes, varianza_expl_acum, marker = 'o', color = 'blue')
plt.plot(range(1, len(varianza_expl_acum)+1), varianza_expl_acum, marker = 'o')
plt.axhline(y = 0.8, color = 'r', linestyle = '--', label = '80% de varianza')
plt.title('Porcentaje de varianza explicada acumulada vs Número de Componentes Principales')
plt.ylabel('Porcentaje de varianza explicada acumulada')
plt.xlabel('Número de componentes principales')
for i in range(len(n_componentes)):
    plt.text(x = n_componentes[i], y = varianza_expl_acum[i] + 0.01, s = f'{varianza_expl_acum[i]:.2f}', ha = 'right')

plt.xticks(ticks = n_componentes)
plt.legend()
plt.grid()
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

In [ ]:
corr_matrix = df_obesity_freq_enc.corr()
print(corr_matrix)
sns.heatmap(corr_matrix)
plt.show()


In [ ]:
proy_pca_freq = pca_pipe_freq.transform(X = df_obesity_freq_enc)
proy_pca_freq = pd.DataFrame(proy_pca_freq)
proy_pca_oh = pca_pipe_oh.transform(X = df_obesity_oh_enc)
proy_pca_oh = pd.DataFrame(proy_pca_oh)
proy_pca_lab = pca_pipe_lab.transform(X = df_obesity_lab_enc)
proy_pca_lab = pd.DataFrame(proy_pca_lab)

plt.figure(figsize = (7, 7))
plt.scatter(proy_pca_freq[0], proy_pca_freq[1])
plt.title('Componente principal 2 vs Componente principal 1')
plt.xlabel('Componente principal 1')
plt.ylabel('Componente principal 2')
plt.tight_layout()
plt.show()

In [ ]:
## K-Means
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 8, init = "k-means++", n_init = 'auto', max_iter = 1000)
y_pred_kmeans = kmeans.fit(proy_pca_freq[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]])
proy_pca_freq['cluster'] = y_pred_kmeans.labels_

plt.figure(figsize=(8, 6))
sns.scatterplot(x = 0, y = 1, hue = 'cluster', data = proy_pca_freq, palette = 'Set1', alpha = 0.7)

plt.title('Clusters obtenidos por K-means en espacio PCA')
plt.xlabel('Componente principal 1')
plt.ylabel('Componente principal 2')
plt.legend(title = 'Cluster')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
## Método del codo
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

inertias = []
k_values = range(1, 30)

for k in k_values:
    kmeans = KMeans(n_clusters = k, init = 'k-means++', n_init = 'auto', max_iter = 1000, random_state = 42)
    kmeans.fit(proy_pca_freq.iloc[:, :12])
    inertias.append(kmeans.inertia_)

## Graficar el método del codo
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertias, 'o-', color='blue')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Suma de errores cuadráticos')
plt.title('Método del codo para determinar k óptimo')
plt.grid(True)
plt.xticks(k_values)
plt.tight_layout()
plt.show()

In [ ]:
##Convertir a formato de base de datos
df_melt_km = proy_pca_freq.melt(id_vars = 'cluster', var_name = 'Variable', value_name = 'Valor')

##Creación de boxplot
plt.figure(figsize = (18, 8))
sns.boxplot(x = 'Variable', y = 'Valor', hue = 'cluster', data = df_melt_km)
plt.title('Distribución por variable y cluster')
plt.legend(title = 'Cluster')
plt.show()

In [ ]:
df_obesity_oh_enc  ## One Hot Encoding Sin Escalar
df_obesity_freq_enc  ## Frequency Encoding Sin Escalar
df_obesity_lab_enc  ## Label Encoding Sin Escalar
df_obesity_oh_enc_scaled  ## One Hot Encoding Escalado
df_obesity_freq_enc_scaled  ## Frequency Encoding Escalado
df_obesity_lab_enc_scaled  ## Label Encoding Escalado
proy_pca_oh  ## Proyección en Componentes Principales One Hot Encoding
proy_pca_freq  ## Proyección en Componentes Principales Frequency Encoding
proy_pca_lab  ## Proyección en Componentes Principales Label Encoding

## UMAP (Uniform Manifold Approximation and Projection)
- Alternativa no lineal a PCA

In [18]:
from umap import UMAP
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D

In [20]:
# Puedes elegir qué codificación usar: One Hot, Frequency, o Label
X_umap_f = UMAP(n_components=2, random_state=42).fit_transform(df_obesity_freq_enc_scaled)
X_umap_oh = UMAP(n_components=2, random_state=42).fit_transform(df_obesity_oh_enc_scaled)
X_umap_lab = UMAP(n_components=2, random_state=42).fit_transform(df_obesity_lab_enc_scaled)

In [21]:
# Guardamos en un DataFrame para visualización
df_umap_f = pd.DataFrame(X_umap_f, columns=['UMAP1', 'UMAP2'])
df_umap_oh = pd.DataFrame(X_umap_oh, columns=['UMAP1', 'UMAP2'])
df_umap_lab = pd.DataFrame(X_umap_lab, columns=['UMAP1', 'UMAP2'])

In [136]:
kmeans_umap = KMeans(n_clusters=2, init="k-means++", n_init='auto', max_iter=1000)

k_umap_f = kmeans_umap.fit_predict(df_umap_f)
k_umap_oh = kmeans_umap.fit_predict(df_umap_oh)
k_umap_lab = kmeans_umap.fit_predict(df_umap_lab)

In [137]:
df_umap_f['cluster'] = k_umap_f
df_umap_oh['cluster'] = k_umap_oh
df_umap_lab['cluster'] = k_umap_lab

In [ ]:
# Estilo general
sns.set(style="whitegrid")

# Crear figura con 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Primer gráfico - df_umap_f
sns.scatterplot(
    data=df_umap_f,
    x='UMAP1', y='UMAP2',
    hue='cluster',
    palette='tab10',
    ax=axes[0],
    legend=False
)
axes[0].set_title('UMAP - Frequency Encoding')

# Segundo gráfico - df_umap_oh
sns.scatterplot(
    data=df_umap_oh,
    x='UMAP1', y='UMAP2',
    hue='cluster',
    palette='tab10',
    ax=axes[1],
    legend=False
)
axes[1].set_title('UMAP - One-Hot Encoding')

# Tercer gráfico - df_umap_lab
sns.scatterplot(
    data=df_umap_lab,
    x='UMAP1', y='UMAP2',
    hue='cluster',
    palette='tab10',
    ax=axes[2],
    legend=True
)
axes[2].set_title('UMAP - Label Encoding')

# Ajustar espacio
plt.tight_layout()
plt.show()


In [144]:
df_encoded = df_umap_lab # Elegir df_umap_f, df_umap_oh o df_umap_lab
# cluster = k_umap_f 
# cluster = k_umap_oh
cluster = k_umap_lab

In [146]:
df_clusters = df_obesity_raw.copy()
df_clusters['cluster'] = cluster  

In [ ]:
# Variables numéricas
numeric_vars = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

# Definir tamaño del gráfico general
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()  # aplanar para iterar fácilmente

# Crear subplot
for i, var in enumerate(numeric_vars):
    sns.boxplot(data=df_clusters, x='cluster', y=var, palette='Set3', ax=axes[i])
    axes[i].set_title(f'{var} por cluster')

plt.suptitle('Boxplots de variables numéricas por cluster', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# Variables categóricas y binarias
cat_vars = [
    'Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC',
    'CAEC', 'CALC', 'MTRANS'
]
fig, axes = plt.subplots(2, 4, figsize=(22, 10))
axes = axes.flatten()

for i, var in enumerate(cat_vars):
    sns.histplot(
        data=df_clusters,
        x='cluster',
        hue=var,
        multiple='fill',      
        shrink=0.8,
        ax=axes[i],
        palette='Set2'
    )
    axes[i].set_title(f'{var}')
    axes[i].set_ylabel('Proporción')
    axes[i].set_xlabel('Cluster')

plt.suptitle('Distribución de variables categóricas/binarias por cluster', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

### UMAP 3D (bad results)

In [145]:
# umap_3d = UMAP(n_components=3, random_state=42)
# umap_3d_proj = umap_3d.fit_transform(df_encoded)  
# kmeans_3d = KMeans(n_clusters=4, init="k-means++", n_init='auto', max_iter=1000)
# k_umap = kmeans_3d.fit_predict(df_encoded)
# # Convertimos en DataFrame para graficar
# df_umap_3d = pd.DataFrame(umap_3d_proj, columns=['UMAP1', 'UMAP2', 'UMAP3'])
# df_umap_3d['cluster'] = k_umap  
# fig = px.scatter_3d(
#     df_umap_3d,
#     x='UMAP1', y='UMAP2', z='UMAP3',
#     color='cluster',
#     title='UMAP 3D con Clusters (KMeans)',
#     opacity=0.8,
#     color_discrete_sequence=px.colors.qualitative.Set1
# )

# fig.show()